# Patient Data Cleaning & Preparation

This notebook performs data cleaning and preparation steps on a healthcare patient dataset to address various data quality issues.

## Step 0: Import Required Libraries and Load Data

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Load the dataset
df = pd.read_csv('patients.csv')

# Display initial data
print(f"Dataset shape: {df.shape}")
df.head()

In [ ]:
# Check the data types and missing values
print("Data types:")
print(df.dtypes)
print("\nMissing values:")
print(df.isna().sum())

## Step 1: Standardizing Date Formats

Convert admission_date and discharge_date columns from inconsistent formats (MM/DD/YYYY, DD-MM-YYYY, etc.) to a consistent YYYY-MM-DD format.

In [ ]:
def standardize_date(date_str):
    """Converts dates from various formats to YYYY-MM-DD"""
    if pd.isna(date_str):
        return pd.NaT
        
    # Remove any whitespace
    date_str = date_str.strip()
    
    # Try different format patterns
    date_formats = ['%m/%d/%Y', '%d-%m-%Y', '%Y/%m/%d', '%m-%d-%Y']
    
    for fmt in date_formats:
        try:
            parsed_date = datetime.strptime(date_str, fmt)
            return parsed_date.strftime('%Y-%m-%d')
        except ValueError:
            continue
    
    return pd.NaT  # If no format matches

# Apply the function to both date columns
df['admission_date'] = df['admission_date'].apply(standardize_date)
df['discharge_date'] = df['discharge_date'].apply(standardize_date)

# Display the standardized dates
print("Standardized Date Formats:")
df[['admission_date', 'discharge_date']].head()

## Step 2: Handling Missing Values

- Fill missing age values with the median age
- Handle missing gender values by marking them as 'Unknown'
- Replace missing bill_amount values with 0

In [ ]:
# Handle missing age values
median_age = df['age'].median()
df['age'] = df['age'].fillna(median_age)
print(f"Filled {df['age'].isna().sum()} missing age values with median age: {median_age}")

# Handle missing gender values
df['gender'] = df['gender'].fillna('Unknown')
print(f"Filled missing gender values with 'Unknown'")

# Handle missing bill_amount values
df['bill_amount'] = df['bill_amount'].fillna(0)
print(f"Filled missing bill_amount values with 0")

# Check remaining missing values
print("\nRemaining missing values:")
print(df.isna().sum())

## Step 3: Standardizing Contact Numbers

Normalize contact number formats to a standard XXX-XXX-XXXX format

In [ ]:
def standardize_phone_number(phone_str):
    """Standardize phone numbers to XXX-XXX-XXXX format"""
    if pd.isna(phone_str):
        return phone_str
        
    # Extract digits only
    digits = re.sub(r'\D', '', str(phone_str))
    
    # Check if we have the expected 10 digits
    if len(digits) == 10:
        return f"{digits[:3]}-{digits[3:6]}-{digits[6:]}"
    else:
        # Return original if format cannot be determined
        return phone_str

# Apply the function to standardize contact numbers
df['contact_number'] = df['contact_number'].apply(standardize_phone_number)

# Display standardized contact numbers
print("Standardized Contact Numbers:")
df[['patient_id', 'name', 'contact_number']].head()

## Step 4: Handling Categorical Data

Convert insurance_status from 'Yes'/'No' to binary values (1/0)

In [ ]:
# Convert insurance_status to binary (1 for Yes, 0 for No)
df['insurance_status_binary'] = df['insurance_status'].map({'Yes': 1, 'No': 0})

# Keep the original column as well
print("Converted Insurance Status to Binary Values:")
df[['patient_id', 'insurance_status', 'insurance_status_binary']].head()

## Step 5: Removing Duplicates

Identify and remove duplicate patient records

In [ ]:
# Check for duplicate records
duplicate_count = df.duplicated().sum()
print(f"Found {duplicate_count} duplicate records")

# Check for potential duplicates based on patient_id
id_duplicate_count = df.duplicated(subset=['patient_id']).sum()
print(f"Found {id_duplicate_count} duplicate patient IDs")

# Remove duplicates if any
df_clean = df.drop_duplicates()
print(f"Original data shape: {df.shape}")
print(f"After removing duplicates: {df_clean.shape}")

## Summary of Data Cleaning

Let's review the transformations performed on the dataset:

In [ ]:
# Display summary statistics of the cleaned data
print("Summary statistics of cleaned data:")
df_clean.describe(include='all').T

In [ ]:
# Save the cleaned dataset
df_clean.to_csv('patients_cleaned.csv', index=False)
print("Cleaned data saved to 'patients_cleaned.csv'")

## Visualization of Cleaned Data

In [ ]:
# Plot distribution of ages
plt.figure(figsize=(10, 6))
sns.histplot(df_clean['age'], bins=10, kde=True)
plt.title('Distribution of Patient Ages')
plt.xlabel('Age')
plt.ylabel('Count')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Distribution of diagnoses
plt.figure(figsize=(12, 6))
diagnosis_counts = df_clean['diagnosis'].value_counts()
sns.barplot(x=diagnosis_counts.index, y=diagnosis_counts.values)
plt.title('Distribution of Diagnoses')
plt.xlabel('Diagnosis')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Relationship between bill amount and insurance status
plt.figure(figsize=(10, 6))
sns.boxplot(x='insurance_status', y='bill_amount', data=df_clean)
plt.title('Bill Amount by Insurance Status')
plt.xlabel('Insurance Status')
plt.ylabel('Bill Amount ($)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()